<a href="https://colab.research.google.com/github/najafiseo/SOV/blob/main/serper_position_and_SOV_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas openpyxl requests

import pandas as pd
import requests
import json
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
import json
import requests
import time

In [ ]:
from google.colab import files
uploaded = files.upload()

excel_filename = list(uploaded.keys())[0]
df = pd.read_excel(excel_filename)

df.head()

In [ ]:
sites_to_check = [
    "idehal.org",
    "blog.idehal.org",
    "digikala.com",
    "torob.com",
    "panasonic-irani.com",
    "panasonicstore.ir",
    "emalls.ir"
]

URL = "https://google.serper.dev/search"

API_KEYS = [
    "کلید اول",
    "کلید دوم",

]

current_key_index = 0
query_count = 0
QUERY_LIMIT_PER_KEY = 1200

def switch_api_key():
    global current_key_index, query_count
    current_key_index = (current_key_index + 1) % len(API_KEYS)
    query_count = 0
    print(f"\n سوئیچ به API جدید: {API_KEYS[current_key_index][:8]}...")

def send_request_with_retry(batch, max_retries=3):
    global query_count

    for attempt in range(max_retries):
        if query_count + len(batch) > QUERY_LIMIT_PER_KEY:
            switch_api_key()

        headers = {
            'X-API-KEY': API_KEYS[current_key_index],
            'Content-Type': 'application/json'
        }

        try:
            response = requests.post(URL, headers=headers, data=json.dumps(batch))
            if response.status_code == 200:
                query_count += len(batch)
                return response.json()
            else:
                print(f"API Error {response.status_code} - سوئیچ کلید")
                switch_api_key()
        except Exception as e:
            print(f" خطا در تلاش {attempt+1}: {e}")
            time.sleep(2)

    print(" تلاش ناموفق، عبور از این batch")
    return None


In [ ]:
keywords = df["کلمه کلیدی"].dropna().tolist()

positions = {site: [] for site in sites_to_check}

batch_size = 100
for i in tqdm(range(0, len(keywords), batch_size)):
    batch_keywords = keywords[i:i+batch_size]

    queries = [{
        "q": keyword,
        "gl": "ir",
        "hl": "fa",
        "num": 20
    } for keyword in batch_keywords]

    response_data = send_request_with_retry(queries)

    if response_data is None:
        print(f"خطا در دریافت اطلاعات batch {i // batch_size + 1}")
        for site in sites_to_check:
            positions[site].extend([None] * len(batch_keywords))
        continue

    for idx, result in enumerate(response_data):
        keyword_results = result.get("organic", [])

        for site in sites_to_check:
            found = False
            for pos, item in enumerate(keyword_results, start=1):
                link = item.get("link", "")
                if site in link:
                    positions[site].append(pos)
                    found = True
                    break
            if not found:
                positions[site].append(None)


In [ ]:
for site in sites_to_check:
    column_name = f"{site}"
    df[column_name] = positions[site]


output_filename = "نتایج_کلمات_کلیدی_با_پوزیشن.xlsx"
df.to_excel(output_filename, index=False)

from google.colab import files
files.download(output_filename)


# SOV

In [ ]:
def weighted_ctr(row, site):
    pos = row[site]
    sv = row["سرچ ولوم"]
    if pd.isna(pos) or pd.isna(sv):
        return 0

    ctr_map = {
        1: 0.276,
        2: 0.158,
        3: 0.11,
        4: 0.084,
        5: 0.063,
        6: 0.049,
        7: 0.039,
        8: 0.033,
        9: 0.027,
        10: 0.024
    }
    try:
        pos = int(pos)
    except:
        return 0

    weight = ctr_map.get(pos, 0)
    return weight * sv


In [ ]:
sov_scores = {}

for site in sites_to_check:
    df[f"{site}_سهم"] = df.apply(lambda row: weighted_ctr(row, site), axis=1)
    sov_scores[site] = df[f"{site}_سهم"].sum()

total = sum(sov_scores.values())
sov_percent = {site: round((score / total) * 100, 2) if total else 0 for site, score in sov_scores.items()}

print(" SOV واقعی بر اساس CTR و سرچ ولوم:")
for site, percent in sov_percent.items():
    print(f"{site}: {percent}%")


In [ ]:
labels = list(sov_percent.keys())
sizes = list(sov_percent.values())

plt.figure(figsize=(8, 8))
plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=140)
plt.axis('equal')
plt.title("SOV")
plt.show()
